# Energy Equation (2D)

## Einführung

## Problem

## Lösung

In [ ]:
using Plots, Interpolations, ExtendableSparse, LinearAlgebra
using GeoModBox.AdvectionEquation.TwoD, GeoModBox.InitialCondition
using GeoModBox.HeatEquation.TwoD, GeoModBox.Tracers.TwoD 
using Base.Threads, Printf

In [ ]:
# Definition numerischer Verfahren =================================== #
# Define Advection Scheme ---
#   1) upwind, 2) slf, 3) semilag, 4) tracers
# Define Diffusion Scheme --- 
#   1) explicit, 2) implicit, 3) CNA, 4) ADI, 5) dc
FD          =   (Method     = (Adv=:?,Diff=:?),)
# Define Initial Condition ---
# Temperature - 
#   1) circle, 2) gaussian, 3) block
# Velocity - 
#   1) RigidBody, 2) ShearCell
Ini         =   (T=:?,V=:?,) 
# -------------------------------------------------------------------- #

In [ ]:
# Plot Einstellungen ================================================= #
Pl  =   (
    inc         =   5,
    sc          =   1.0*(100*(60*60*24*365.15)),        
    Minc        =   1, 
    Msz         =   0.2,
)
# Animationssettings ---
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("07_2D_EnergyEquation_",Ini.T,"_",Ini.V,
                        "_",FD.Method.Adv,"_",FD.Method.Diff)
save_fig    =   1
# -------------------------------------------------------------------- #

In [ ]:
# Model Konstanten =================================================== #
M   =   (
    xmin    =   ?,
    xmax    =   ?,
    ymin    =   ?,
    ymax    =   ?,
)
# -------------------------------------------------------------------- #
# Physical Parameters ================================================ #
P       = ( 
    k       =   ?,              #   Thermal Conductivity [ W/m/K ]
    cp      =   ?,           #   Specific Heat Capacity [ J/kg/K ]
    ρ       =   ?,           #   Density [ kg/m^3 ]
    K0      =   ?,         #   Kelvin at 0 °C
    Q0      =   ?               #   Heat production rate
)
P1      = (
    κ       =   ?,   #   Thermal Diffusivity [ m^2/s ] 
)
P       =   merge(P,P1)
# -------------------------------------------------------------------- #

In [ ]:
# Numerische Konstanten ============================================== #
NC  =   (
    x       =   ?,        # Number of horizontal centroids
    y       =   ?,        # Number of vertical centroids
)
NV  =   (
    x       =   ?,  # Number of horizontal vertices
    y       =   ?,  # Number of vertical vertices
)
Δ   =   (
    x   =   ?,
    y   =   ?,
)
# -------------------------------------------------------------------- #

In [ ]:
# Erstellung des Gitters ============================================= #
x   =   (
    c       =   LinRange(?),
    ce      =   LinRange(?),
    v       =   LinRange(?)
)
y       = (
    c       =   LinRange(?),
    ce      =   LinRange(?),
    v       =   LinRange(?),
)
x1      =   ( 
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# -------------------------------------------------------------------- #

In [ ]:
# Felder Initialisierung ============================================= #
D       =   (
    Q       =   zeros(Float64,(NC.x,NC.y)),
    T       =   zeros(Float64,(NC.x,NC.y)),
    T0      =   zeros(Float64,(NC.x,NC.y)),
    T_ex    =   zeros(Float64,(NC.x+2,NC.y+2)),
    T_exo   =   zeros(Float64,(NC.x+2,NC.y+2)),
    ρ       =   zeros(Float64,(NC.x,NC.y)),
    cp      =   zeros(Float64,(NC.x,NC.y)),
    vx      =   zeros(Float64,(NV.x,NV.y+1)),
    vy      =   zeros(Float64,(NV.x+1,NV.y)),    
    vxc     =   zeros(Float64,(NC.x,NC.y)),
    vyc     =   zeros(Float64,(NC.x,NC.y)),
    vc      =   zeros(Float64,(NC.x,NC.y)),
    wt      =   zeros(Float64,(NC.x,NC.y)),
    wtv     =   zeros(Float64,(NV...)),
    Tmax    =   [0.0],
    Tmin    =   [0.0],
    Tmean   =   [0.0],
)
# -------------------------------------------------------------------- #

In [ ]:
# Anfangsbedingungen ================================================= #
# Temperatur ---
IniTemperature!(?)
if FD.Method.Adv==:slf
    D.T_exo    .=  D.T_ex
end
# Heat production rate ---
@. D.Q  =   P.Q0
@. D.ρ  =   P.ρ
# Velocity ---
# RBR - maximum(v) = 0.5 cm/a
IniVelocity!(?)        # [ m/s ]
# Upscale velocity ---
D.vx    .*=     20.0
D.vy    .*=     20.0
# Get the velocity on the centroids ---
@threads for i = 1:NC.x 
    for j = 1:NC.y
        D.vxc[i,j]  = ?
        D.vyc[i,j]  = ?
    end
end
@. D.vc        = sqrt(D.vxc^2 + D.vyc^2)
# -------------------------------------------------------------------- #

In [ ]:
# Boundary Conditions ================================================ #
BC     = (type    = (W=:?, E=:?, 
                    N=:?, S=:?),
        val     = (W=?,E=?,
                    N=?,S=?))
# -------------------------------------------------------------------- #

In [ ]:
# Linear Equations =================================================== #
if FD.Method.Diff==:implicit || FD.Method.Diff==:CNA
    Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
    ndof    =   maximum(Num.T)        
    if FD.Method.Diff==:CNA
        K1      =   ExtendableSparseMatrix(ndof,ndof)
        K2      =   ExtendableSparseMatrix(ndof,ndof)
    else
        K       =   ExtendableSparseMatrix(ndof,ndof)
    end
    rhs     =   zeros(ndof)
elseif FD.Method.Diff==:dc
    niter       =   10
    ϵ           =   1e-10
    @. D.ρ      =   P.ρ
    @. D.cp     =   P.cp
    k           =   (x=zeros(NC.x+1,NC.y), y=zeros(NC.x,NC.y+1))
    @. k.x      =   P.k
    @. k.y      =   P.k
    Num         =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
    ndof        =   maximum(Num.T)
    K           =   ExtendableSparseMatrix(ndof,ndof)
    R           =   zeros(NC.x,NC.y)
    ∂T          =   (∂x=zeros(NC.x+1, NC.y), ∂y=zeros(NC.x, NC.y+1))
    q           =   (x=zeros(NC.x+1, NC.y), y=zeros(NC.x, NC.y+1))
end
# -------------------------------------------------------------------- #

In [ ]:
# Time =============================================================== #
T   =   ( 
    tmax    =   [0.0], 
    Δfacc   =   1.0,        # Courant time factor, i.e. dtfac*dt_courant
    Δfacd   =   1.0,        # Diffusion time factor, i.e. dtfac*dt_diff 
    Δ       =   [0.0],
    Δc      =   [0.0],      # Courant time step
    Δd      =   [0.0],      # Diffusion time stability criterion
)
T.tmax[1]   =   π*((M.xmax-M.xmin)-Δ.x)/maximum(D.vc)   # t = U/v [ s ]
T.Δc[1]     =   ?
T.Δd[1]     =   ?

T.Δ[1]      =   ?

nt          =   ceil(Int,T.tmax[1]/T.Δ[1])
# -------------------------------------------------------------------- #

In [ ]:
# Tracer Advection =================================================== #
if FD.Method.Adv==:tracers 
    # Tracer Initialization ---
    nmx,nmy     =   3,3
    noise       =   1
    nmark       =   nmx*nmy*NC.x*NC.y
    Aparam      =   :thermal
    MPC         =   (
        c       =   zeros(Float64,(NC.x,NC.y)),
        v       =   zeros(Float64,(NV.x,NV.y)),
        th      =   zeros(Float64,(nthreads(),NC.x,NC.y)),
        thv     =   zeros(Float64,(nthreads(),NV.x,NV.y)),
    )
    MPC1        = (
        PG_th   =   [similar(D.ρ) for _ = 1:nthreads()],    # per thread
        PV_th   =   [similar(D.wtv) for _ = 1:nthreads()],   # per thread
        wt_th   =   [similar(D.wt) for _ = 1:nthreads()],   # per thread
        wtv_th  =   [similar(D.wtv) for _ = 1:nthreads()],  # per thread
    )
    MPC     =   merge(MPC,MPC1)
    Ma      =   IniTracer2D(Aparam,nmx,nmy,Δ,M,NC,noise,0,0)
    # RK4 weights ---
    rkw     =   1.0/6.0*[1.0 2.0 2.0 1.0]   # for averaging
    rkv     =   1.0/2.0*[1.0 1.0 2.0 2.0]   # for time stepping
    # Interpolate on tracers ---
    @threads for k = 1:nmark
        Ma.T[k] =   FromCtoM(D.T_ex, k, Ma, x, y, Δ, NC)
    end
    # Count marker per cell ---
    CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV,1)
end
# -------------------------------------------------------------------- #

In [ ]:
# Visualize initial condition ======================================== #
p = heatmap(?)
quiver!(?)
if save_fig == 1
    Plots.frame(anim)
elseif save_fig == 0
    display(p)
end
# -------------------------------------------------------------------- #

In [ ]:
# Zeitschleife ======================================================= #
for i=2:nt
    @printf("Time step: #%04d\n ",i)
    
    # Loesung Advektionsgleichung ---
    if FD.Method.Adv==:upwind
        ?
    elseif FD.Method.Adv==:slf
        ?       
    elseif FD.Method.Adv==:semilag
        ?    
    elseif FD.Method.Adv==:tracers
        # NEED TO GET RID OF THE NAN'S IN D.T FIRST!
        ## Interpolate from grid on tracers ---
        @threads for k = 1:nmark
           Ma.T[k] =   FromCtoM(D.T_ex, k, Ma, x, y, Δ, NC)
        end
        # Advect tracers ---
        AdvectTracer2D(Ma,nmark,D,x,y,T.Δ[1],Δ,NC,rkw,rkv,1)
        CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV,i)
        
        # Interpolate from tracers to grid ---
        Markers2Cells(Ma,nmark,MPC.PG_th,D.T,MPC.wt_th,D.wt,x,y,Δ,Aparam,0)
        D.T_ex[2:end-1,2:end-1]     .= D.T
    end

    # Loesung Diffusionsgleichung ---
    if FD.Method.Diff==:explicit
        ?
    elseif FD.Method.Diff==:implicit
        ?
    elseif FD.Method.Diff==:CNA
        ?
    elseif FD.Method.Diff==:ADI
        ?
    elseif FD.Method.Diff==:dc
        D.T0    .=  D.T
        for iter = 1:niter
            # Evaluate residual
            ComputeResiduals2D!(R, D.T, D.T_ex, D.T0, ∂T, q, D.ρ, D.cp, k, BC, Δ, T.Δ[1])
            # @printf("||R|| = %1.4e\n", norm(R)/length(R))
            norm(R)/length(R) < ϵ ? break : nothing
            # Assemble linear system
            K  = AssembleMatrix2D(D.ρ, D.cp, k, BC, Num, NC, Δ, T.Δ[1])
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT = -(Kc\R[:])
            # Update temperature
            @. D.T += δT[Num.T]
        end
        #D.T0    .= D.T
    end

    @printf("ΔT = %4.4f\n\n",abs((D.Tmax[1]-maximum(filter(!isnan,D.T)))/D.Tmax[1]*100))

    # Plot Solution ---
    if mod(i,10) == 0 || i == nt
        p = heatmap(?)
        quiver!(?)
        if save_fig == 1
            Plots.frame(anim)
        elseif save_fig == 0
            display(p)                        
        end
    end

end # End Time loop ---
# -------------------------------------------------------------------- #

In [ ]:
# Save Animation ===================================================== #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
    foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
elseif save_fig == 0
    display(plot(p))
end
# -------------------------------------------------------------------- #